In [1]:
# components for one episode
from scai.custom_chat_models.crfm import crfmChatLLM # model
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

# meta prompt 
from scai.modules.meta_prompt.base import MetaPromptModel # model
from scai.modules.meta_prompt.prompts import META_PROMPTS # prompts

# user 
from scai.modules.user.base import UserModel # model
from scai.modules.user.prompts import USER_PROMPTS # prompts

# assistant
from scai.modules.assistant.base import AssistantModel # model
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS # prompts
from scai.modules.assistant.models import AssistantPrompt # prompt

# task
from scai.modules.task.prompts import TASK_PROMPTS

In [2]:
# sample task
task_prompt = TASK_PROMPTS["task_prompt_1"]
# sample users 
user_prompt = USER_PROMPTS["user_prompt_1"]
# sample assistant
assistant_prompt = ASSISTANT_PROMPTS["assistant_prompt_1"]
# sample meta prompt
meta_prompt = META_PROMPTS["meta_prompt_1"]

In [7]:
# initialise buffer 
buffer = CustomConversationBufferWindowMemory(system_k=2, 
                                              chat_k=2, 
                                              user_k=2, 
                                              assistant_k=1,
                                              ) # how many turns are stored will be used

system_message = "You are a helpful AI assistant"
chat_history = ""

# initialise task
task = task_prompt.content
print("TASK")
print(task)
print("=====================================")

# initialise user 
print("USER")
print(user_prompt.content.format(persona=user_prompt.persona, 
                                 task=task, 
                                 chat_history=chat_history, 
                                 max_tokens=user_prompt.max_tokens))
print("=====================================")

# initialise assistant
print("ASSISTANT")
print(assistant_prompt.content.format(system_message=system_message, 
                                      chat_history=chat_history, 
                                      max_tokens=assistant_prompt.max_tokens))
buffer.save_context(system={"system": system_message})
print("=====================================")

print("BUFFER")
print(buffer.full_memory.messages)

TASK
Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic:
"What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?"

USER
1. You are collaborating with a few other Users and AI Assistant(s) on a <task>. You will adopt a Persona <persona> that guides your preferences and responses throughout your conversation.
2. You are given the <chat_history> of the conversation so far. If it is empty, your response should be based on the <persona> and <task> and it should be in the form of a query for the AI Assistant to help you with the task. Otherwise, you should respond based on the <persona>, <task>, and <chat_history>, and provide feedback to the AI Assistant and other Users.

Persona: You're a serious Wikipedia editor committed to the facts. If an 

In [8]:
user_model = UserModel(llm='chat_llm')
user_response = user_model.run(user_prompt=user_prompt, 
                               task_prompt=task_prompt, 
                               buffer=buffer)

print(user_response)

System: 1. You are collaborating with a few other Users and AI Assistant(s) on a <task>. You will adopt a Persona <persona> that guides your preferences and responses throughout your conversation.
2. You are given the <chat_history> of the conversation so far. If it is empty, your response should be based on the <persona> and <task> and it should be in the form of a query for the AI Assistant to help you with the task. Otherwise, you should respond based on the <persona>, <task>, and <chat_history>, and provide feedback to the AI Assistant and other Users.

Persona: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. 

Task: Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic:
"What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses

In [9]:
assistant_model = AssistantModel(llm='chat_llm')
assistant_response = assistant_model.run(assistant_prompt=assistant_prompt, buffer=buffer)

print(assistant_response)

System: system: You are a helpful AI assistant

Your response should be at most 100 tokens long.
Response:


In [10]:
meta_model = MetaPromptModel(llm='chat_llm')
meta_response = meta_model.run(meta_prompt=meta_prompt, buffer=buffer)

print(meta_response)

System: Assistant has just had the below interaction with the User. Both Assistant and User followed their system message closely to answer a question together. 
        
Your job is to critique Assistant's performance and provide new system message so that Assistant can correctly and quickly respond in the future.

#### Chat History Starts #### 



#### Chat History Ends #### 

Your first task is to critique Assistan's performance: 
1. What could Assistant have done better to satisfy User? 
2. Was Assistant effective in answering the question posed? 
3. Was Assistant receptive to User's feedback? 
4. Did Assistant satisfy User as correctly and quickly as possible? 

You should indicate this feedback with “Critique: ...”.

Your next task is to revise Assistant's system message. Older system messages including previous revisions are shown below. 

#### Assistant's old system message(s) Start #### 

system: You are a helpful AI assistant

#### Assistant's old system message(s) End #### 


In [ ]:
# # prompting a custom chat model
# from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

# chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
#                    crfm_api_key=CRFM_API_KEY, 
#                    max_tokens=100, # need to be careful with this one
#                    num_completions=1,
#                    request_timeout=10,
#                    verbose=False,
#                    temperature=0.9,
#                    )